# Imports

In [15]:
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
from typing import Union
import xarray as xr
import matplotlib.pyplot as plt
from ipywidgets import interact

# Path to ESA Landcover datasets on Jasmin
from src.constants import ESA_LANDCOVER_DIR, GWS_DATA_DIR, UTM35N
from src.models.fragmentation_baseline import fragmentation_analysis

# Load data

In [2]:
# Parse land cover classes:
landcover_classes = ESA_LANDCOVER_DIR / "ESACCI-LC-Legend.csv"
classes = pd.read_csv(landcover_classes, delimiter=";", index_col=0)
landcover_classes = classes.index.tolist()

# Parse land cover data:
esa_cci_landcovers = {year: GWS_DATA_DIR / "esa_cci_rois" / f"esa_cci_{year}_chernobyl.tif"  
                      for year in tqdm(range(1992, 2016))}

# Parse region of interest data
rois = gpd.read_file(GWS_DATA_DIR / "chernobyl_exclusion_zone_v1.geojson")[:-1]


  0%|          | 0/24 [00:00<?, ?it/s]

In [3]:
rois

,name,area (km2),geometry
0,bbox,44022.318370,"POLYGON ((28.40773 50.54449, 31.41977 50.54449..."
1,E+60,17608.538086,"POLYGON ((28.42221 51.16929, 28.42178 51.17077..."
2,E+30,12731.935189,"POLYGON ((28.84467 51.21708, 28.84425 51.21856..."
3,Chernobyl Exclusion Zone,2597.858793,GEOMETRYCOLLECTION (MULTIPOLYGON (((29.26795 5...
4,Polesie Exclusion Zone,2216.675202,GEOMETRYCOLLECTION (MULTIPOLYGON (((29.46530 5...


# Perform fragmentation analysis

Warning, the computation below takes about 5 minutes for the Chernobyl & Polesia exclusion zone (`rois.iloc[3:5]`)

In [4]:
frag_stats = fragmentation_analysis(landcover_data = esa_cci_landcovers,
                       landcover_classes = landcover_classes,
                       target_crs = UTM35N,
                       target_x_res = 300,
                       target_y_res = 300,
                       rois = rois.iloc[3:5],
                       rois_index_col="name",
                       save_path = None)

  0%|          | 0/24 [00:00<?, ?it/s]INFO: Analysing data for 1992
/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/pylandstats/landscape.py:912: RuntimeWarning: Class 61 has less than 2 patches. Euclidean-nearest-neighbor might contain nan values
  warnings.warn(
/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/pylandstats/landscape.py:912: RuntimeWarning: Class 200 has less than 2 patches. Euclidean-nearest-neighbor might contain nan values
  warnings.warn(
/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/pylandstats/landscape.py:912: RuntimeWarning: Class 61 has less than 2 patches. Euclidean-nearest-neighbor might contain nan values
  warnings.warn(
/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/pylandstats/landscape.py:912: RuntimeWarning: Class 200 has less than 2 patches. Euclidean-nearest-neighbor might contain nan values
  warnings.warn(
/home/users/svm/Code/gtc-biodiversity/env/lib/python3.

In [5]:
frag_stats

<xarray.DataArray (zone: 1, landcover_class: 38, metric: 45, date: 24)>
array([[[[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[4.98308185e+04, 4.98308185e+04, 4.98308185e+04, ...,
          5.24067272e+04, 5.22631225e+04, 5.22631225e+04],
         [1.92416996e+01, 1.92416996e+01, 1.92416996e+01, ...,
          2.02363624e+01, 2.01809108e+01, 2.01809108e+01],
         [2.36000000e+02, 2.36000000e+02, 2.36000000e+02, ...,
          1.97000000e+02, 1.98000000e+02, 1.98000000e+02],
...
         [1.19834711e+03, 1.19834711e+03, 1.19834711e+03, ...,
          1.97796935e+03, 1.97796935e+03, 1.97796935e+03],
         [2.59697632e+02, 2.59697632e+02, 2.59697632e+02, ...,
          3.60452361e+02, 4.10052113e+02, 4.10052113e+02],
         [3.38099732e+01, 3.38099732e+01, 3.38099732e+01, ...,
          4.59547490e+01, 4.94057924e+01, 4.94057924e+01]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]]]])
Coordinates:
  * zone             (zone) <U24 'Chernobyl Exclusion Zone'
  * landcover_class  (landcover_class) int64 0 10 11 12 20 ... 201 202 210 220
  * metric           (metric) <U29 'total_area' ... 'euclidean_nearest_neighb...
  * date             (date) int64 1992 1993 1994 1995 ... 2012 2013 2014 2015

# Display

In [76]:
def display_class_metric(data: xr.DataArray,
                         zone: str, 
                         landcover_class: Union[int, str], 
                         metrics: str,
                         fig = None, 
                         **kwargs):
    
    '''Plot the temporal variation of class metrics for different class value'''
    # Input parsing        
    if isinstance(metrics, str):
        metrics = [metrics]
        
    if isinstance(landcover_class, str):
        # Deal with string labels
        assert landcover_class in classes.LCCOwnLabel.values
        landcover_class = classes[classes.LCCOwnLabel == landcover_class].index[0]
    
    # Create figure if necessary
    if not fig:
        plt.figure(figsize=(8,5))
        
    for metric in metrics:
        metric_name = metric.replace("_", " ").capitalize()
        class_name = classes.loc[landcover_class].LCCOwnLabel
        
        plot_title = f"{metric_name}: {class_name}({landcover_class})\n {zone}"
        
        plt.title(plot_title)
        
        plt.plot(data.date, 
                 data.loc[{"zone" : zone, 
                           "landcover_class": landcover_class, 
                           "metric": metric
                          }],
                 linestyle="dashed",
                 marker=".")
        plt.show()

In [81]:
@interact(
    zone = frag_stats.coords["zone"].data.tolist(),
    landcover_class = [(name.LCCOwnLabel, idx) for idx, name in classes.iterrows()],
    metrics=frag_stats.coords["metric"].data.tolist(), value="total_area")
def idisplay(zone ="Chernobyl Exclusion Zone",
             landcover_class = 10,
             metrics="total_area"
            ):
    try:
        display_class_metric(frag_stats, zone, landcover_class, metrics)
        plt.show()
    except:
        print("No data")

interactive(children=(Dropdown(description='zone', options=('Chernobyl Exclusion Zone',), value='Chernobyl Exc…